In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pyl
import seaborn as sns

%matplotlib inline

In [2]:
from glob import glob
import cv2
import os
import shutil
import math

#### Data extraction and preparation

In [ ]:
# Extracting data

# Root directory
Dir = 'Datasets\\animals\\'
No_of_images = {}

for dir in os.listdir(Dir):
    No_of_images[dir] = len(os.listdir(os.path.join(Dir, dir)))

# Displaying number of images per class available
No_of_images.items()

In [ ]:
# Function to split data into training, validation and test set

def data_selection(X, split):
    if not os.path.exists('.\\'+X):
        os.mkdir('.\\'+X)
        
        for dir in os.listdir(Dir):
            os.makedirs('.\\'+X+'\\'+dir)
            for img in np.random.choice(a = os.listdir(os.path.join(Dir, dir)),
                                       size = (math.floor(split*No_of_images[dir])-5),
                                       replace = False):
                O = os.path.join(Dir, dir, img)
                D = os.path.join('.\\'+X, dir)
                shutil.copy(O,D)
                os.remove(O)
                
    else:
        print(f'{X} is unavailable')

In [ ]:
# Applying data-split funtion

data_selection('Train', 0.7)
data_selection('Valid', 0.15)
data_selection('Test', 0.15)

#### Data preprocessing

In [3]:
from keras.applications.mobilenet import preprocess_input
from keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras

In [4]:
# Function to preprocess training data

def Train_processing(path):
    image = ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2,
                               preprocessing_function = preprocess_input,
                              horizontal_flip = True)
    Image = image.flow_from_directory(directory = path, target_size = (224,224),
                                     batch_size = 12, class_mode = 'categorical')
    return Image

In [5]:
# Function to preprocess other data sets

def main_processing(path):
    image = ImageDataGenerator(preprocessing_function = preprocess_input)
    Image = image.flow_from_directory(directory = path, target_size = (224,224),
                                     batch_size = 12, class_mode = 'categorical')
    return Image

In [ ]:
# Applying preprocessing functions

train = 'C:\\Users\\PC\\Documents\\Train'
valid = 'C:\\Users\\PC\\Documents\\Valid'
test = 'C:\\Users\\PC\\Documents\\Test'

Train = Train_processing(train)
Valid = main_processing(valid)
Test = main_processing(test)

#### Model importation and modification

In [ ]:
from keras.applications.mobilenet import MobileNet

In [ ]:
# Instantiating imported model

Base_model = MobileNet(input_shape = (224, 224, 3), include_top = False)

In [ ]:
# Model info

Base_model.summary()

In [ ]:
for i in Base_model.layers:
    i.trainable = False

In [ ]:
# Adding Flattening and final dense layer to model

Output = Flatten()(Base_model.output)
Out = Dense(4, activation = 'softmax')(Output)

model = Model(Base_model.input, Out)

In [ ]:
# Modified model info

model.summary()

In [ ]:
# Compiling model

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'rmsprop', 
             metrics = ['accuracy'])

In [ ]:
# Fitting model

hist_=model.fit_generator(Train, steps_per_epoch = 12, epochs = 25,
                   validation_data = Valid, validation_steps = 16)

#### Model evaluation and deployment

In [ ]:
# model evaluation

model.evaluate_generator(Valid)

In [ ]:
H = hist_.history

H.keys()

In [ ]:
# Accuracy plot

plt.plot(H['accuracy'], c = 'r')

In [ ]:
# Loss plot

plt.plot(H['loss'])

In [ ]:
# Prediction on test set

Pred_img = Cnn_model_1.predict(Test)
N_pred = np.argmax(Pred_img, axis = 1)
N_pred

A new function is created to classify any random image of a cheetah, leopard, lion or tiger.
This just requires the path to the image and the model classifies this image

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
def Image_classes_prediction(path, model):
    img = load_img(path, target_size = (224, 224))
    in_arr = img_to_array(img)
    in_arr = preprocess_input(in_arr)
    plt.imshow(in_arr)
    in_arr = np.expand_dims(in_arr, axis = 0)
    
    Pred = model.predict(in_arr)
    Pred_ = np.argmax(Pred)
    
    if Pred_ == 0:
        print('Cheetah')
    elif Pred_ == 1:
        print('Leopard')
    elif Pred_ == 2:
        print('Lion')
    else:
        print('Tiger')
        
    return Pred_

In [ ]:
Leo = ''
Image_classes_prediction(Leo, model)